# Explore kalman filter



This notebook explores how to run the Kalman filter on mock data 

In [1]:
%load_ext autoreload
%autoreload 2

# 1. Get the data to pass to the filter 

In [2]:
import sys
import glob 
sys.path.append('../src')
from data_loader import LoadWidebandPulsarData

In [20]:

import pandas as pd 

datadir = '../data/IPTA_MockDataChallenge/IPTA_Challenge1_open/Challenge_Data/Dataset2/'


# Get all .par files in the directory
par_files = sorted(glob.glob(datadir + '*.par'))
tim_files = sorted(glob.glob(datadir + '*.tim'))

assert len(par_files) == len(tim_files) 


# Combine par_files and tim_files into pairs
file_pairs = list(zip(par_files, tim_files))



dfs = []
dfs_meta = []
total_num_rows = 0 
i = 0
# Check we can load the files with no errors
for par_file, tim_file in file_pairs[0:2]:
    psr = LoadWidebandPulsarData.read_par_tim(par_file, tim_file)

    print(psr.M_matrix.shape)
    print(psr.fitpars)

    df = pd.DataFrame({'toas': psr.toas, f'residuals_{i}': psr.residuals})

    df_meta = pd.DataFrame({'name': [psr.name], f'dim_M': [psr.M_matrix.shape[-1]],f'RA': [psr.RA],f'DEC': [psr.DEC]})

    dfs.append(df)
    dfs_meta.append(df_meta)

    total_num_rows += len(df)
    i += 1
 






(130, 8)
['Offset', 'RAJ', 'DECJ', 'F0', 'F1', 'PMRA', 'PMDEC', 'PX']
(130, 11)
['Offset', 'RAJ', 'DECJ', 'F0', 'F1', 'PX', 'PB', 'A1', 'TASC', 'EPS1', 'EPS2']


In [21]:
dfs_meta 

[         name  dim_M        RA       DEC
 0  J0030+0451      8  0.132895  0.084841,
          name  dim_M      RA       DEC
 0  J0218+4232     11  0.6026  0.742431]

In [22]:
from functools import reduce 

merged_df = reduce(lambda left, right: pd.merge(left, right, on='toas', how='outer'), dfs)


In [23]:
merged_df 

,toas,residuals_0,residuals_1
0,4.580106e+09,3.380995e-06,NaN
1,4.580106e+09,NaN,-9.349598e-07
2,4.581370e+09,2.576195e-06,NaN
3,4.581370e+09,NaN,7.212309e-06
4,4.582755e+09,1.867198e-06,NaN
...,...,...,...
255,4.731184e+09,-9.911458e-07,NaN
256,4.732025e+09,NaN,-1.214947e-05
257,4.732025e+09,-8.987281e-07,NaN
258,4.733030e+09,-1.237823e-06,NaN


In [24]:
combined_df = pd.concat(dfs_meta, ignore_index=True)


In [25]:
combined_df

,name,dim_M,RA,DEC
0,J0030+0451,8,0.132895,0.084841
1,J0218+4232,11,0.602600,0.742431


# 2. Specify the model to use with the filter 

In [33]:
from models import StochasticGWBackgroundModel

In [34]:
psr_information = combined_df

In [36]:
combined_df

,name,dim_M,RA,DEC
0,J0030+0451,8,0.132895,0.084841
1,J0218+4232,11,0.602600,0.742431


In [60]:
model = StochasticGWBackgroundModel(psr_information)

In [71]:

θ = {'dt': 0.50}
F_array = model.F_matrix(θ)

In [72]:
F_array

array([[1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 1. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
    

In [68]:
import numpy as np

#Helper function
def print_non_zero_blocks(matrix):
    rows, cols = matrix.shape
    visited = np.zeros((rows, cols), dtype=bool)
    
    for i in range(rows):
        for j in range(cols):
            if matrix[i, j] != 0 and not visited[i, j]:
                # Find the block
                block_rows, block_cols = [], []
                for k in range(i, rows):
                    if matrix[k, j] != 0:
                        block_rows.append(k)
                    else:
                        break
                for l in range(j, cols):
                    if matrix[i, l] != 0:
                        block_cols.append(l)
                    else:
                        break
                
                # Mark the block as visited
                for r in block_rows:
                    for c in block_cols:
                        visited[r, c] = True
                
                # Print the block
                block = matrix[np.ix_(block_rows, block_cols)]
                print(f"Block at ({i}, {j}):")
                print(block)
                print()

In [69]:
print_non_zero_blocks(F_array)

Block at (0, 0):
[[1. 1.]]

Block at (1, 1):
[[1. 1.]]

Block at (2, 2):
[[1.]]

Block at (3, 3):
[[1.]]

Block at (4, 4):
[[1.]]

Block at (5, 5):
[[1.]]

Block at (6, 6):
[[1.]]

Block at (7, 7):
[[1.]]

Block at (8, 8):
[[1.]]

Block at (9, 9):
[[1.]]

Block at (10, 10):
[[1.]]

Block at (11, 11):
[[1.]]

Block at (12, 12):
[[1.]]

Block at (13, 13):
[[1. 1.]]

Block at (14, 14):
[[1. 1.]]

Block at (15, 15):
[[1.]]

Block at (16, 16):
[[1.]]

Block at (17, 17):
[[1.]]

Block at (18, 18):
[[1.]]

Block at (19, 19):
[[1.]]

Block at (20, 20):
[[1.]]

Block at (21, 21):
[[1.]]

Block at (22, 22):
[[1.]]

Block at (23, 23):
[[1.]]

Block at (24, 24):
[[1.]]

Block at (25, 25):
[[1.]]

Block at (26, 26):
[[1.]]

Block at (27, 27):
[[1.]]

Block at (28, 28):
[[1.]]



# 3. Now run the filter

In [4]:
from kalman_filter import KalmanFilter

In [ ]:


observations = merged_df 
model = 


x0 = 
P0 = #maybe should be part of the model?
self, model, observations, x0, P0